In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Nina
[nltk_data]     Kim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import twint
import pandas as pd
import nest_asyncio
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import seaborn as sns

In [3]:
#cleaning
import re
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords


In [4]:
nest_asyncio.apply()

In [5]:
company_search = {"Zara":"Zara OR ZARA OR zara", "Gucci":"Gucci OR GUCCI",
                 "Patagonia":"Patagonia OR patagonia OR PATAGONIA"}

In [6]:
def twintConfig(date_from, date_to, search_string):
    c = twint.Config()
    c.Search = search_string[1]
    c.Since = date_from
    c.Until = date_to
    c.Lang = 'en'
    c.Pandas = True
    twint.run.Search(c)

In [7]:
since = "2022-09-01"
until = "2022-10-01"

In [8]:
def Run_Twint(search_vals):
    
    #set empty dataframe for join
    out_df= pd.DataFrame()
    
    for company in search_vals.items():
        print ("running for search item: "+company[0]+"\n")

        #run twint
        twintConfig(since, until, company)
        
        #get dataframe from twint output
        tweets_df = twint.storage.panda.Tweets_df
        
        #join Dataframes and create 'Bank' column
        tweets_df["Company"]= company[0]
        out_df = pd.concat([out_df,tweets_df])
        
    return out_df

In [ ]:
tweets_df = Run_Twint(company_search)

In [11]:
len(tweets_df)

138203

In [12]:
tweets_df.head(2)

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,Company
0,1575998008491094016,1575998008491094016,1.664582e+12,2022-10-01 00:56:20,+0000,,Check out this listing I just added to my #Pos...,en,"[poshmark, shopmycloset]",[],...,,,,,[],,,,,Zara
1,1575997511759634434,1575997511759634434,1.664582e+12,2022-10-01 00:54:22,+0000,,zara larsson chewed and spat out poster girl l...,en,[],[],...,,,,,[],,,,,Zara


In [13]:
tweets_df = tweets_df[ ~tweets_df["username"].str.lower().str.contains('zara|gucci|patagonia',regex = True)]

In [14]:
tweets_df = tweets_df.drop_duplicates(subset=['date',"tweet","Company"],keep="first")

In [15]:
tweets_df.reset_index(inplace=True)
tweets_df.drop("index",axis =1,inplace=True)

In [16]:
tweets_df["language"].unique()
tweets_df = tweets_df[tweets_df["language"].isin([ 'und', 'en'])]

In [17]:
len(tweets_df)

136438

In [18]:
def clean_text(text):  
    pat1 = r'@[^ ]+'                   #@signs
    pat2 = r'https?://[A-Za-z0-9./]+'  #links
    pat3 = r'\'s'                      #floating s's
    pat4 = r'\#\w+'                     # hashtags
    pat5 = r'&amp '
    pat6 = r'[^A-Za-z\s]'         #remove non-alphabet
    combined_pat = r'|'.join((pat1, pat2,pat3,pat4,pat5, pat6))
    text = re.sub(combined_pat,"",text).lower()
    return text.strip()

In [19]:
%%time
tweets_df["cleaned_tweet"] = tweets_df["tweet"].apply(clean_text)

Wall time: 1.61 s


In [20]:
#drop empty rows
tweets_df = tweets_df [ ~(tweets_df["tweet"] =="")]

In [21]:
tweets_df["cleaned_tweet"].head()

0    check out this listing i just added to my  clo...
1    zara larsson chewed and spat out poster girl l...
2    check out this listing i just added to my  clo...
3                                    good morning zara
4    check out this listing i just added to my  clo...
Name: cleaned_tweet, dtype: object

In [22]:
tweets_df.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,Company,cleaned_tweet
0,1575998008491094016,1575998008491094016,1.664582e+12,2022-10-01 00:56:20,+0000,,Check out this listing I just added to my #Pos...,en,"[poshmark, shopmycloset]",[],...,,,,[],,,,,Zara,check out this listing i just added to my clo...
1,1575997511759634434,1575997511759634434,1.664582e+12,2022-10-01 00:54:22,+0000,,zara larsson chewed and spat out poster girl l...,en,[],[],...,,,,[],,,,,Zara,zara larsson chewed and spat out poster girl l...
2,1575997238412980224,1575997238412980224,1.664582e+12,2022-10-01 00:53:16,+0000,,Check out this listing I just added to my #Pos...,en,"[poshmark, shopmycloset]",[],...,,,,[],,,,,Zara,check out this listing i just added to my clo...
3,1575996936380764160,1575988601267290113,1.664582e+12,2022-10-01 00:52:04,+0000,,@KhadeejahMalana good morning zara,en,[],[],...,,,,[],,,,,Zara,good morning zara
4,1575996600228671489,1575996600228671489,1.664582e+12,2022-10-01 00:50:44,+0000,,Check out this listing I just added to my #Pos...,en,"[poshmark, shopmycloset]",[],...,,,,[],,,,,Zara,check out this listing i just added to my clo...


In [23]:
tweets_df.to_csv('tweets_data.csv')